## Mount GDrive, Define Paths

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# On Google Colab:
data_dir = '/content/gdrive/My Drive/All datasets/Semantic segmentation'
runs_dir = data_dir + '/runs/segmented_im'
masks_dir = data_dir + '/runs/masks'

# Path to vgg model
vgg_path = data_dir + '/vgg'



## Define Libraries and Helper Functions

In [0]:
import os.path
import tensorflow as tf
import helper
import warnings
import cv2
import numpy as np
from distutils.version import LooseVersion
import re
import random
import numpy as np
import os.path
import scipy.misc
import shutil
import zipfile
import time
import tensorflow as tf
import cv2
from glob import glob
from urllib.request import urlretrieve
from tqdm import tqdm
from matplotlib import pyplot as plt

In [4]:
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.13.1
Default GPU Device: /device:GPU:0


In [5]:
# List the city subfolders to train the NN on
train_cities = [
  'aachen',
  'bochum',
  'bremen', 
  'cologne',
  'darmstadt', 
  'dusseldorf', 
  'erfurt',
  'hamburg',
  'hanover',
  'jena',  
  'krefeld',
  'monchengladbach',
  'strasbourg',  
  'stuttgart',
  'tubingen',
  'zurich'                     
] 

# Initialize array for storing training image paths
image_paths = []

# Initialize dictionary for mapping training images to their annotations
label_paths = {}

for i in range(len(train_cities)):
  
  # grab all the image file paths
  image_paths.extend(glob(os.path.join(data_dir,'leftImg8bit/train',train_cities[i],'*.png')))

  # create the dictionary entries for these files and add it to the top level dictionary
  label_paths_temp = {
    re.sub(r'gtFine_color', 'leftImg8bit', os.path.basename(path)): path
    for path in glob(os.path.join(data_dir,'gtFine_trainvaltest/gtFine/train',train_cities[i],'*.png'))}
  label_paths.update(label_paths_temp)

  
print('Number of training images:',len(image_paths))
print('Number of training labels:',len(label_paths))



Number of training images: 2737
Number of training labels: 8214


In [6]:
# CityScape RGB Colors for Active Classes
road_color = np.array([128, 64, 128,255])
car_color = np.array([0, 0, 142,255])
person_color = np.array([220, 20, 60,255])
trafficL_color = np.array([250, 170, 30,255])
vegetation_color = np.array([107,142,35,255])

print(road_color)

[128  64 128 255]


In [0]:
#helper modules
class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num


In [0]:
#helper modules
def gen_batch_function(data_folder, image_shape):
    """
    Generate function to create batches of training data
    :param data_folder: Path to folder that contains all the datasets
    :param image_shape: Tuple - Shape of image
    :return:
    """
    #print(data_folder)
    def get_batches_fn(batch_size):
        print('inside get_batches_fn')
        """
        Create batches of training data
        :param batch_size: Batch Size
        :return: Batches of training data
        """
#         image_paths = glob(os.path.join(data_folder, 'leftImg8bit/train/aachen','*.png'))
#         print(len(image_paths))
#         label_paths = {
#             re.sub(r'gtFine_color', 'leftImg8bit', os.path.basename(path)): path
#             for path in glob(os.path.join(data_folder,'gtFine_trainvaltest/gtFine/train/aachen', '*.png'))}
        
#         road_color = np.array([128, 64, 128, 255])
#         car_color = np.array([0, 0, 142, 255])
#         person_color = np.array([220, 20, 60, 255])
#         trafficL_color = np.array([250, 170, 30, 255])
#         vegetation_color = np.array([107,142,35,255])
        

        random.shuffle(image_paths)
        for batch_i in range(0, len(image_paths), batch_size):
            images = []
            gt_images = []
            for image_file in image_paths[batch_i:batch_i+batch_size]:
                
                image = scipy.misc.imread(image_file)
                
                
                gt_image_file = label_paths[os.path.basename(image_file)]
                gt_image = scipy.misc.imread(gt_image_file)
                
                
                # Resizing the Images
                image = scipy.misc.imresize(image, image_shape)
                gt_image = scipy.misc.imresize(gt_image, image_shape)
                
                # Encoding the ground_truth tensor
                gt_road = np.all(gt_image == road_color, axis=2)
                gt_road = gt_road.reshape(*gt_road.shape, 1) 
                
                
                gt_car = np.all(gt_image == car_color, axis=2)
                gt_car = gt_car.reshape(*gt_car.shape, 1) 
                
                gt_person = np.all(gt_image == person_color, axis=2) 
                gt_person = gt_person.reshape(*gt_person.shape, 1) 
                
                gt_trafficL = np.all(gt_image == trafficL_color, axis=2) 
                gt_trafficL = gt_trafficL.reshape(*gt_trafficL.shape, 1) 
                
                gt_vegetation = np.all(gt_image == vegetation_color, axis=2) 
                gt_vegetation = gt_vegetation.reshape(*gt_vegetation.shape, 1) 
                
                gt_image = np.concatenate((gt_road,gt_car,gt_person,gt_trafficL,gt_vegetation), axis=2)
                gt_bg = np.all(gt_image == 0, axis=2)
                gt_bg = gt_bg.reshape(*gt_bg.shape, 1) 
                gt_image = np.concatenate((gt_image,gt_bg), axis=2)
                
                
                images.append(image)
                
                gt_images.append(gt_image)
              
            yield np.array(images), np.array(gt_images)
    
    return get_batches_fn


In [0]:
#helper modules
def gen_test_output(sess, logits, keep_prob, image_pl, data_folder, image_shape):
    """
    Generate test output using the test images
    :param sess: TF session
    :param logits: TF Tensor for the logits
    :param keep_prob: TF Placeholder for the dropout keep robability
    :param image_pl: TF Placeholder for the image placeholder
    :param data_folder: Path to the folder that contains the datasets
    :param image_shape: Tuple - Shape of image
    :return: Output for for each test image
    """
    for image_file in glob(os.path.join(data_folder, '*.png')):
        image = scipy.misc.imresize(scipy.misc.imread(image_file), image_shape)

        im_softmax = sess.run(
            [tf.nn.softmax(logits)],
            {keep_prob: 1.0, image_pl: [image]})
        
        #Road
        im_softmax_r = im_softmax[0][:, 0].reshape(image_shape[0], image_shape[1])
        segmentation_r = (im_softmax_r > 0.5).reshape(image_shape[0], image_shape[1], 1)
        mask = np.dot(segmentation_r, np.array([[128, 64, 128, 128]]))
        
        #Car
        im_softmax_c = im_softmax[0][:, 1].reshape(image_shape[0], image_shape[1])
        segmentation_c = (im_softmax_c > 0.5).reshape(image_shape[0], image_shape[1], 1)
        mask = mask + np.dot(segmentation_c, np.array([[0, 0, 142, 128]]))
        
        #Person
        im_softmax_p = im_softmax[0][:, 2].reshape(image_shape[0], image_shape[1])
        segmentation_p = (im_softmax_p > 0.5).reshape(image_shape[0], image_shape[1], 1)
        mask = mask + np.dot(segmentation_p, np.array([[220, 20, 60, 128]]))
        
        #Traffic Light
        im_softmax_t = im_softmax[0][:, 3].reshape(image_shape[0], image_shape[1])
        segmentation_t = (im_softmax_t > 0.5).reshape(image_shape[0], image_shape[1], 1)
        mask = mask + np.dot(segmentation_t, np.array([[250, 170, 30, 128]]))
        
        #Vegetation
        im_softmax_v = im_softmax[0][:, 4].reshape(image_shape[0], image_shape[1])
        segmentation_v = (im_softmax_v > 0.5).reshape(image_shape[0], image_shape[1], 1)
        mask = mask + np.dot(segmentation_v, np.array([[107,142,35,128]]))
        
        mask = scipy.misc.toimage(mask, mode="RGBA")
        mask_name = os.path.basename(image_file)
        scipy.misc.imsave(os.path.join(masks_dir, mask_name), mask)
        
        street_im = scipy.misc.toimage(image)
        street_im.paste(mask, box=None, mask=mask)

        yield os.path.basename(image_file), np.array(street_im)


In [0]:
# #helper modules
# def gen_test_output(sess, logits, keep_prob, image_pl, data_folder, image_shape):
#     """
#     Generate test output using the test images
#     :param sess: TF session
#     :param logits: TF Tensor for the logits
#     :param keep_prob: TF Placeholder for the dropout keep robability
#     :param image_pl: TF Placeholder for the image placeholder
#     :param data_folder: Path to the folder that contains the datasets
#     :param image_shape: Tuple - Shape of image
#     :return: Output for for each test image
#     """
#     for image_file in glob(os.path.join(data_folder, '*.png')):
#         image = scipy.misc.imresize(scipy.misc.imread(image_file), image_shape)

#         im_softmax = sess.run(
#             [tf.nn.softmax(logits)],
#             {keep_prob: 1.0, image_pl: [image]})
        
#         #Road
#         im_softmax_r = im_softmax[0][:, 0].reshape(image_shape[0], image_shape[1])
#         segmentation_r = (im_softmax_r > 0.5).reshape(image_shape[0], image_shape[1], 1)
#         mask = np.dot(segmentation_r, np.array([[128, 64, 128, 128]]))
        
#         #Car
#         im_softmax_c = im_softmax[0][:, 1].reshape(image_shape[0], image_shape[1])
#         segmentation_c = (im_softmax_c > 0.5).reshape(image_shape[0], image_shape[1], 1)
#         mask = mask + np.dot(segmentation_c, np.array([[0, 0, 142, 128]]))
        
#         #Person
#         im_softmax_p = im_softmax[0][:, 2].reshape(image_shape[0], image_shape[1])
#         segmentation_p = (im_softmax_p > 0.5).reshape(image_shape[0], image_shape[1], 1)
#         mask = mask + np.dot(segmentation_p, np.array([[220, 20, 60, 128]]))
        
#         #Traffic Light
#         im_softmax_t = im_softmax[0][:, 3].reshape(image_shape[0], image_shape[1])
#         segmentation_t = (im_softmax_t > 0.5).reshape(image_shape[0], image_shape[1], 1)
#         mask = mask + np.dot(segmentation_t, np.array([[250, 170, 30, 128]]))
        
#         #Vegetation
#         im_softmax_v = im_softmax[0][:, 4].reshape(image_shape[0], image_shape[1])
#         segmentation_v = (im_softmax_v > 0.5).reshape(image_shape[0], image_shape[1], 1)
#         mask = mask + np.dot(segmentation_v, np.array([[107,142,35,255]]))
        
#         mask = scipy.misc.toimage(mask, mode="RGBA")
#         street_im = scipy.misc.toimage(image)
#         street_im.paste(mask, box=None, mask=mask)

#         yield os.path.basename(image_file), np.array(street_im)


In [0]:
#helper modules
def save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image):
    # Make folder for current run
    output_dir = os.path.join(runs_dir)
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)

#     if os.path.exists(masks_dir):
#         shutil.rmtree(masks_dir)
#     os.makedirs(masks_dir)
    
    # Run NN on test images and save them to HD
    #print('Training Finished. Saving test images to: {}'.format(output_dir))
    image_outputs = gen_test_output(
        sess, logits, keep_prob, input_image, os.path.join(data_dir,'leftImg8bit/test_images'), image_shape)
    for name, image in image_outputs:
        scipy.misc.imsave(os.path.join(output_dir, name), image)
        #scipy.misc.imsave(os.path.join(masks_dir, name), mask)
        


In [0]:
#helper modules
def gen_video_output(sess, logits, keep_prob, image_pl, video_file, image_shape):

    cap = cv2.VideoCapture(video_file)
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    filename = 'semantic_output.mp4'
    out = cv2.VideoWriter(filename, fourcc, 29.0, (512,256))

    while True:
        ret, frame = cap.read()
        if frame is None:
            break
        image = scipy.misc.imresize(frame, image_shape)

        im_softmax = sess.run(
            [tf.nn.softmax(logits)],
            {keep_prob: 1.0, image_pl: [image]})
        
        #Road
        im_softmax_r = im_softmax[0][:, 0].reshape(image_shape[0], image_shape[1])
        segmentation_r = (im_softmax_r > 0.5).reshape(image_shape[0], image_shape[1], 1)
        mask = np.dot(segmentation_r, np.array([[128, 64, 128, 200]]))
        
        #Car
        im_softmax_c = im_softmax[0][:, 1].reshape(image_shape[0], image_shape[1])
        segmentation_c = (im_softmax_c > 0.5).reshape(image_shape[0], image_shape[1], 1)
        mask = mask + np.dot(segmentation_c, np.array([[220, 20, 60, 200]]))
        
        #Person
        im_softmax_p = im_softmax[0][:, 2].reshape(image_shape[0], image_shape[1])
        segmentation_p = (im_softmax_p > 0.5).reshape(image_shape[0], image_shape[1], 1)
        mask = mask + np.dot(segmentation_p, np.array([[0, 0, 142, 200]]))
       
        #Traffic Light
        im_softmax_t = im_softmax[0][:, 3].reshape(image_shape[0], image_shape[1])
        segmentation_t = (im_softmax_t > 0.5).reshape(image_shape[0], image_shape[1], 1)
        mask = mask + np.dot(segmentation_t, np.array([[250, 170, 30, 200]]))
        
        #Vegetation
        im_softmax_v = im_softmax[0][:, 4].reshape(image_shape[0], image_shape[1])
        segmentation_v = (im_softmax_v > 0.5).reshape(image_shape[0], image_shape[1], 1)
        mask = mask + np.dot(segmentation_v, np.array([[107,142,35,255]]))
        
        
        mask = scipy.misc.toimage(mask, mode="RGBA")
        street_im = scipy.misc.toimage(image)
        street_im.paste(mask, box=None, mask=mask)
        
        output = np.array(street_im)

        out.write(output)

    cap.release()
    out.release()
    cv2.destroyAllWindows()
       

## Define Pretrained CNN VGG-16 Network

In [0]:
def maybe_download_pretrained_vgg(data_dir):
    
    """
    Download and extract pretrained vgg model if it doesn't exist
    :param data_dir: Directory to download the model to
    """
    vgg_filename = 'vgg.zip'
    vgg_path = os.path.join(data_dir, 'vgg')
    vgg_files = [
        os.path.join(vgg_path, 'variables/variables.data-00000-of-00001'),
        os.path.join(vgg_path, 'variables/variables.index'),
        os.path.join(vgg_path, 'saved_model.pb')]

    missing_vgg_files = [vgg_file for vgg_file in vgg_files if not os.path.exists(vgg_file)]
    if missing_vgg_files:
        # Clean vgg dir
        if os.path.exists(vgg_path):
            shutil.rmtree(vgg_path)
        os.makedirs(vgg_path)

        # Download vgg
        print('Downloading pre-trained vgg model...')
        with DLProgress(unit='B', unit_scale=True, miniters=1) as pbar:
            urlretrieve(
                'https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/vgg.zip',
                os.path.join(vgg_path, vgg_filename),
                pbar.hook)

        # Extract vgg
        print('Extracting model...')
        zip_ref = zipfile.ZipFile(os.path.join(vgg_path, vgg_filename), 'r')
        zip_ref.extractall(data_dir)
        zip_ref.close()

        # Remove zip file to save space
        os.remove(os.path.join(vgg_path, vgg_filename))


In [0]:
# Load VGG16
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    # Load the model & weight 
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph = tf.get_default_graph()
    image_input = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3 = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4 = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7 = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    return image_input, keep_prob, layer3, layer4, layer7

## Define the Skip Level Architecture

In [0]:
# Define the layers
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    conv_1x1_layer7 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, padding='SAME', 
                                       kernel_initializer=tf.random_normal_initializer(stddev=0.001),
                                       kernel_regularizer=tf.contrib.layers.l2_regularizer(Reg))
    
    deconv_layer7 = tf.layers.conv2d_transpose(conv_1x1_layer7, num_classes, 4, 2, padding='SAME', 
                                               kernel_initializer=tf.random_normal_initializer(stddev=0.001),
                                               kernel_regularizer=tf.contrib.layers.l2_regularizer(Reg))
    
    conv_1x1_layer4 = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, padding='SAME',
                                       kernel_initializer=tf.random_normal_initializer(stddev=0.001),
                                       kernel_regularizer=tf.contrib.layers.l2_regularizer(Reg))
    
    skip_connect1 = tf.add (deconv_layer7, conv_1x1_layer4)
    
    deconv_skip_connect1 = tf.layers.conv2d_transpose(skip_connect1, num_classes, 4, 2, padding='SAME', 
                                               kernel_initializer=tf.random_normal_initializer(stddev=0.001),
                                               kernel_regularizer=tf.contrib.layers.l2_regularizer(Reg))
    
    conv_1x1_layer3 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, padding='SAME', 
                                       kernel_initializer=tf.random_normal_initializer(stddev=0.001),
                                       kernel_regularizer=tf.contrib.layers.l2_regularizer(Reg))
    
    skip_connect2 = tf.add (deconv_skip_connect1, conv_1x1_layer3)
    
    output = tf.layers.conv2d_transpose(skip_connect2, num_classes, 16, 8, padding='SAME', 
                                        kernel_initializer=tf.random_normal_initializer(stddev=0.001),
                                               kernel_regularizer=tf.contrib.layers.l2_regularizer(Reg)) 
    
    return output

In [0]:
# Optimizer
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    logits = tf.reshape(nn_last_layer, (-1,num_classes))
    print('logits:',logits.shape)
    labels = tf.reshape(correct_label, (-1,num_classes))
    print('labels:',labels.shape)
    cross_entropy_loss = tf.reduce_mean (tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy_loss)
    
    return logits, train_op, cross_entropy_loss

In [0]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate,image_shape,logits):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    #Train function 
    
    
    for epoch in range(epochs):
        if epoch<=3:
          tf=0.01
        elif epoch>3 and epoch<=6:
          tf=0.001
        elif epoch>6:
          tf=0.0001
      
        
        print("EPOCH: {}...".format(epoch+1))
        for image, label in get_batches_fn(batch_size):
            _, loss = sess.run([train_op, cross_entropy_loss], feed_dict = {input_image: image, correct_label: label, keep_prob: 0.5, learning_rate: tf})
        print("LOSS: {}".format(loss))
        
        n=3
        if(epoch%n==0):
          # Save inference data using helper.save_inference_samples
          save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)
        

## Train the Model

In [0]:
def run():
 
    num_classes = 6
    epochs = 10
    batch_size = 16
    image_shape = (256, 512) #1/4 Resize
        
    #pretrained vgg model
    
    maybe_download_pretrained_vgg(data_dir)

    with tf.Session() as sess:
        
        # Create function to get batches
        get_batches_fn = gen_batch_function(os.path.join(data_dir), image_shape)
        
        
        # Building NN using load_vgg, layers, and optimize function
        
        image_input, keep_prob, layer3, layer4, layer7 = load_vgg(sess, vgg_path)
        
        output = layers(layer3, layer4, layer7, num_classes)
        
        
        #TF Placeholder for correct_label,learning_rate
        correct_label = tf.placeholder(dtype = tf.int32, shape = [None, None, None, num_classes])
        learning_rate = tf.placeholder(dtype = tf.float32)
        logits, train_op, cross_entropy_loss = optimize(output, correct_label, learning_rate, num_classes)

        # Training NN using the train_nn function
        sess.run(tf.global_variables_initializer())
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, image_input, correct_label, keep_prob, learning_rate,image_shape,logits)
        
        # Save inference data using helper.save_inference_samples
        save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, image_input)
        
        print ("Finished..")

In [0]:
# Define Variables
Reg = 0.001
loss=0

run()

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/All datasets/Semantic segmentation/vgg/variables/variables

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.conv2d_transpose instead.
logits: (?, 6)
labels: (?, 6)
Instructions for updating:

Future major versions of TensorFlow will a

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


LOSS: 1.192747950553894


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4

EPOCH: 2...
inside get_batches_fn
LOSS: 1.6813852787017822
EPOCH: 3...
inside get_batches_fn
LOSS: 0.850299596786499
EPOCH: 4...
inside get_batches_fn
LOSS: 1.4541983604431152
EPOCH: 5...
inside get_batches_fn
LOSS: 1.5174472332000732
EPOCH: 6...
inside get_batches_fn
LOSS: 0.9829670190811157
EPOCH: 7...
inside get_batches_fn


In [0]:
data_folder='/content/gdrive/My Drive/All datasets/Semantic segmentation'
image_paths = glob(os.path.join(data_folder, 'leftImg8bit/train/aachen','*.png'))

In [0]:
image_paths
for image_file in image_paths:
  #print(os.path.basename(image_file))
  


In [0]:
#data_folder='/content/gdrive/My Drive/All datasets/Semantic segmentation/gtFine_trainvaltest/gtFine/train/aachen'
label_paths = {
            re.sub(r'gtFine_color', 'leftImg8bit', os.path.basename(path)): path
            for path in glob(os.path.join(data_folder,'gtFine_trainvaltest/gtFine/train/aachen', '*.png'))}

In [0]:
label_paths